In [1]:
key = ""

In [ ]:
from openai import OpenAI


def process_text(user_input):

    # Внутренний промпт, который будет добавлен к пользовательскому вводу
    system_prompt = """
        внутренние инструкции        
        """

    # Объединяем внутренний промпт с пользовательским вводом
    combined_text = f"{system_prompt}\n{user_input}"

    client = OpenAI(api_key=key)

    # Отправляем запрос к модели OpenAI
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Убедитесь, что используете правильное название модели
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ],
        max_tokens=150,  # Опционально: ограничение на количество токенов в ответе
        temperature=0.3,  # Опционально: настройка креативности ответа
    )
    response = completion.choices[0].message
    answer = response.content
    return answer

In [3]:
user_input = "в принципе, нормальный ресторанчик при отеле. после выезда ходили сюда на бизнесланч. за 390 р. салат или суп плюс основное блюдо. борщ был немного пересолен, а вот гречотто с биточками очень омномном. за 90 р. во время бизнесланча также дали полноценный кофе латте, по вкусу вполне приличный. единственный минус бутылочку воды, которая уже включена в стоимость ланча, пришлось отдельно попросить принести и то принесли только одну на два заказанных ланча. но все равно 5 баллов, ибо вкусно и недорого."
output = process_text(user_input)
if output:
    print("Ответ модели:")
    print(output)

Ответ модели:
бизнес-ланч, салат, суп, основное блюдо, гречотто, биточки, кофе латте, вкусная еда, приличный вкус, недорогая цена


In [23]:
import pandas as pd

df = pd.read_csv(
    r"C:\Users\zPad\OneDrive\Documents\Study\MIPT\Study\Project_2024_12\test-3\review_flamp_yandex_v6.csv",
    sep=",",
)
df

,place_id,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source
0,4504127908348140,495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,"в принципе, нормальный ресторанчик при отеле. ...",5.0,flamp
1,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"есть такой ресторан на юге москвы, по названию...",5.0,flamp
2,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"раньше было лучше, потом было хуже, сейчас сре...",4.0,flamp
3,4504127908348160,Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,сегодня рестораны москвы все чаще предлагают н...,5.0,flamp
4,4504127908348296,Восточные вечера,раковарня,Москва,улица Свердлова,55.822870,37.969245,1.0,1,1.0,1500.0,отвратительно. больше ничего не могу сказать. ...,1.0,flamp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47829,70000001091778281,караоке-клуб маска,ресторан,Москва,"улица Маросейка, 7/8",55.757592,37.632880,5.0,5,0.0,NaN,хороший караоке клуб! бронировали vip. все отл...,5.0,yandex
47830,70000001091778282,шаурма 1001,ресторан,Москва,"Братиславская улица, 31к1",55.652684,37.767690,1.0,1,0.0,NaN,"залетели ночью с друзьями, разбудили работника...",1.0,yandex
47831,70000001091778283,petruccio. pizza&pasta,ресторан,Москва,"улица Юных Ленинцев, 52",55.699796,37.765354,5.0,3,0.0,NaN,очень вкусная пицца. нравится их система лояль...,5.0,yandex
47832,70000001091778283,petruccio. pizza&pasta,ресторан,Москва,"улица Юных Ленинцев, 52",55.699796,37.765354,5.0,3,0.0,NaN,"пицца детям понравилась, интерьер очень интере...",5.0,yandex


In [5]:
positive_reviews = df[(df["rating_decimal"] >= 4) & (df["rating"] >= 4)]

# 2. Группировка и объединение отзывов
grouped_reviews = (
    positive_reviews.groupby("place_id")["review"].apply("|".join).reset_index()
)
grouped_reviews.rename(columns={"review": "combined_review"}, inplace=True)
grouped_reviews.head(2)

,place_id,combined_review
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ..."
1,4504127908348160,"есть такой ресторан на юге москвы, по названию..."


In [8]:
place_id = 4504127908349335

test_review = grouped_reviews[grouped_reviews["place_id"] == place_id][
    "combined_review"
]
for combined_review in test_review:
    # Разбиваем combined_review, если необходимо
    if len(combined_review) > 10000:
        review_groups = split_review(combined_review)
    else:
        review_groups = [combined_review]

    combined_features = []

    for group in review_groups:
        try:
            features = process_text(group)
            if features:  # Проверяем, что features не пустой
                combined_features.append(features)
        except Exception as e:
            # Обработка возможных ошибок при вызове process_text
            combined_features.append(f"Ошибка обработки: {e}")

    # Очистка и удаление дубликатов
    unique_features = clean_features(combined_features)


display(len(combined_review))
display(combined_review)
display(unique_features)

73411

'starbucks для меня особенное место. место праздничного настроения, теплой атмосферы, умопомрачительных ароматов кофе и пирожных. я люблю абсолютно все а особенно карамель маккиато и горячий шоколад. . особенно приятно согреваться там зимой. я редко пью кофе, но если пью, то только в старбаксе, а потому каждый такой поход для меня маленький праздник. желаю всем побольше таких праздников!|starbucks ну кто же его не знает. долгое время я обходила стороной, потому что не люблю большие сети, но старбакс это чтото особенное. и дело не в том хороший или плохой там кофе. дело в атмосфере, минуте тишины, желании рисовать и разглядывать все и вся вокруг. дело в том, что спрашивают твое имя, что подарят купон, если что не так, что можно часами сидеть на удобных диванах и мечтать. просто старбакс это уютно, мне там нравится|starbucks стал уже легендой! просто обожаю их кофе, как первый раз в штатах попробовала так и влюбилась!|starbucks это уже символ эпохи. каждый самоуважающий хипстер обязан вз

'free wifi, авторский латте, аккуратность, аккуратный, ароматный, ароматный кофе, ароматный латте, атмосфера, атмосферное место, бесплатные добавки, бесподобный, бесподобный латте, бодрит, бодрящее кофе, бодрящий, бодрящий аромат, бодрящий кофе, большой ассортимент, быстрое обслуживание, быстрое приготовление, вежливые сотрудники, вежливый, вежливый персонал, веселый персонал, вкус кофе, вкусная еда, вкуснейшее кофе, вкуснейшие, вкуснейшие десерты, вкуснейший латте, вкусное моко, вкусное печенье, вкусности, вкусные блюда, вкусные десерты, вкусные и ароматные напитки, вкусные коктейли, вкусные напитки, вкусные омлеты, вкусные сладости, вкусный, вкусный кофе, вкусный морковный торт, вкусный шоколадный торт, внимательное обслуживание, внимательное отношение, внимательный персонал, воздушный, возможность перекусить, восхитительный, восхитительный чизкейк, высокое качество, городская романтика, горячий согревающий кофе, горячий шоколад, демократичный, десерты, доброжелательный персонал, дом

In [ ]:
unique_features

In [10]:
import os
import time
import pandas as pd
from tqdm import tqdm


# Функция для разбивки текста на группы не более 10000 символов по символу '|'
def split_review(review, max_length=10000, delimiter="|"):
    parts = review.split(delimiter)
    groups = []
    current_group = ""

    for part in parts:
        addition = delimiter + part if current_group else part
        if len(current_group) + len(addition) > max_length:
            if current_group:
                groups.append(current_group)
            current_group = part
            if len(current_group) > max_length:
                # Если один сегмент превышает max_length, добавляем его отдельно
                groups.append(current_group)
                current_group = ""
        else:
            current_group += addition

    if current_group:
        groups.append(current_group)

    return groups


# Функция для очистки и удаления дубликатов из списка фич
def clean_features(combined_features):
    feature_set = set()
    for feature_str in combined_features:
        # Удаляем кодовые блоки и переводы строк
        feature_str = feature_str.replace("```", "").replace("\n", "").strip()
        # Разделяем по запятой
        features = feature_str.split(",")
        for feat in features:
            feat = feat.strip()
            if feat:
                feature_set.add(feat)
    # Сортируем для консистентности
    return ", ".join(sorted(feature_set))


# Путь к файлу для сохранения промежуточных результатов
save_file = "features_df_2.csv"

# Проверяем, существует ли файл сохранения
if os.path.exists(save_file):
    # Загружаем существующие данные
    existing_df = pd.read_csv(save_file)
    processed_place_ids = set(existing_df["place_id"].tolist())
    print(f"Загружено {len(existing_df)} уже обработанных записей.")
else:
    # Инициализируем пустой DataFrame
    existing_df = pd.DataFrame(columns=["place_id", "combined_review", "features"])
    processed_place_ids = set()
    print("Файл сохранения не найден. Начинаем обработку с нуля.")

# Инициализируем список для новых результатов
new_features = []

# Инициализируем прогресс-бар
with tqdm(total=grouped_reviews.shape[0], desc="Обработка отзывов") as pbar:
    for index, row in grouped_reviews.iterrows():
        place_id = row["place_id"]
        combined_review = row["combined_review"]

        if place_id in processed_place_ids:
            pbar.update(1)
            continue

        # Разбиваем combined_review, если необходимо
        if len(combined_review) > 10000:
            review_groups = split_review(combined_review)
        else:
            review_groups = [combined_review]

        combined_features = []

        for group in review_groups:
            try:
                features = process_text(group)
                if features:  # Проверяем, что features не пустой
                    combined_features.append(features)
            except Exception as e:
                # Обработка возможных ошибок при вызове process_text
                combined_features.append(f"Ошибка обработки: {e}")

        # Очистка и удаление дубликатов
        unique_features = clean_features(combined_features)

        # Добавление результата в список новых фич
        new_features.append(
            {
                "place_id": place_id,
                "combined_review": combined_review,
                "features": unique_features,
            }
        )

        # Добавляем place_id в множество обработанных
        processed_place_ids.add(place_id)

        # Каждые 10 итераций сохраняем результаты
        if len(new_features) >= 10:
            # Преобразуем список в DataFrame
            temp_df = pd.DataFrame(new_features)
            # Записываем в CSV файл
            temp_df.to_csv(
                save_file, mode="a", index=False, header=not os.path.exists(save_file)
            )
            # Очищаем список новых фич
            new_features = []
            # Пауза для предотвращения превышения лимитов API
            time.sleep(1)

        # Обновляем прогресс-бар
        pbar.update(1)

# После завершения цикла сохраняем оставшиеся фичи
if new_features:
    temp_df = pd.DataFrame(new_features)
    temp_df.to_csv(
        save_file, mode="a", index=False, header=not os.path.exists(save_file)
    )

Файл сохранения не найден. Начинаем обработку с нуля.


Обработка отзывов: 100%|██████████| 9761/9761 [4:24:13<00:00,  1.62s/it]   


In [ ]:
import os
import time
import pandas as pd

# Путь к файлу для сохранения промежуточных результатов
save_file = "features_df_2.csv"

# Проверяем, существует ли файл сохранения
if os.path.exists(save_file):
    # Загружаем существующие данные
    existing_df = pd.read_csv(save_file)
    processed_place_ids = set(existing_df["place_id"].tolist())
    print(f"Загружено {len(existing_df)} уже обработанных записей.")
else:
    # Инициализируем пустой DataFrame
    existing_df = pd.DataFrame(columns=["place_id", "combined_review", "features"])
    processed_place_ids = set()
    print("Файл сохранения не найден. Начинаем обработку с нуля.")

# Инициализируем список для новых результатов
new_features = []

# Итерация по сгруппированным отзывам
for index, row in grouped_reviews.iterrows():
    place_id = row["place_id"]
    combined_review = row["combined_review"]

    if place_id in processed_place_ids:
        print(f"place_id {place_id} уже обработан. Пропускаем.")
        continue

    print(f"Обрабатывается place_id: {place_id}")

    # Получение фич с помощью process_text
    features = process_text(combined_review)

    # Добавление результата в список новых фич
    new_features.append(
        {"place_id": place_id, "combined_review": combined_review, "features": features}
    )

    # Добавляем place_id в множество обработанных
    processed_place_ids.add(place_id)

    # Каждые 10 итераций сохраняем результаты
    if len(new_features) >= 10:
        # Преобразуем список в DataFrame
        temp_df = pd.DataFrame(new_features)
        # Записываем в CSV файл
        temp_df.to_csv(
            save_file, mode="a", index=False, header=not os.path.exists(save_file)
        )
        print(f"Сохранено {len(new_features)} записей.")
        # Очищаем список новых фич
        new_features = []
        # Пауза для предотвращения превышения лимитов API
        time.sleep(1)

# После завершения цикла сохраняем оставшиеся фичи
if new_features:
    temp_df = pd.DataFrame(new_features)
    temp_df.to_csv(
        save_file, mode="a", index=False, header=not os.path.exists(save_file)
    )
    print(f"Сохранено {len(new_features)} записей.")

# Загрузка итогового DataFrame из сохранённого файла
features_df = pd.read_csv(save_file)
print("Обработка завершена. Итоговый DataFrame содержит", len(features_df), "записей.")

In [24]:
import pandas as pd

dff = pd.read_csv(
    r"C:\Users\zPad\OneDrive\Documents\Study\MIPT\Study\Project_2024_12\feature_extraction\features_df_2.csv",
    sep=",",
)
dff

,place_id,combined_review,features
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к..."
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело..."
2,4504127908348342,прекрасные завтраки от 195 р. есть постные блю...,"завтраки, кофе с соевым молоком, милые надписи..."
3,4504127908348384,крейзи! именно так можно сказать о данном заве...,"бесплатные концерты, веселая атмосфера, винило..."
4,4504127908348455,уютный семейный итальянский ресторан с плетено...,"арматная, атмосфера, баклажаны, буррата, зелен..."
...,...,...,...
9756,70000001091778278,"заказала тортик, очень вкусный, свежий, заказы...","вкуснейший, вкусные десерты, вкусный, вкусняшк..."
9757,70000001091778279,"расположен на 4 этаже трц калейдоскоп, на фудк...","ассортимент, известная специфика, сетевой рест..."
9758,70000001091778280,"в целом вкусно, но раньше шаурмист прям большу...","большая шаурма, вкусная шаурма, вкусно, лагман"
9759,70000001091778281,бронировали vip комнату 4.02 на 6 человек и не...,"10% скидка, безупречное обслуживание, беспрово..."


In [25]:
df6 = df.copy()
df6.drop_duplicates(subset=["place_id"], inplace=True)
dff = dff.merge(df6, on="place_id", how="left")
dff

,place_id,combined_review,features,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к...",495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,"в принципе, нормальный ресторанчик при отеле. ...",5.0,flamp
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело...",Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,"есть такой ресторан на юге москвы, по названию...",5.0,flamp
2,4504127908348342,прекрасные завтраки от 195 р. есть постные блю...,"завтраки, кофе с соевым молоком, милые надписи...",Coffee,кофейня,Москва,"Душинская улица, 7с1",55.745250,37.706002,5.0,1,0.0,1500.0,прекрасные завтраки от 195 р. есть постные блю...,5.0,flamp
3,4504127908348384,крейзи! именно так можно сказать о данном заве...,"бесплатные концерты, веселая атмосфера, винило...",Bourbon street,пивной ресторан,Москва,"Большой Златоустинский переулок, 7с1",55.758709,37.632634,4.9,5,0.0,NaN,крейзи! именно так можно сказать о данном заве...,5.0,flamp
4,4504127908348455,уютный семейный итальянский ресторан с плетено...,"арматная, атмосфера, баклажаны, буррата, зелен...",Cantinetta Antinori,ресторан,Москва,"Денежный переулок, 20",55.745428,37.586787,5.0,1,0.0,5000.0,уютный семейный итальянский ресторан с плетено...,5.0,flamp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9756,70000001091778278,"заказала тортик, очень вкусный, свежий, заказы...","вкуснейший, вкусные десерты, вкусный, вкусняшк...",особенный наполеон,ресторан,Москва,"Мосфильмовская улица, 53",55.704864,37.497450,2.0,7,0.0,NaN,брала торт на др ребенка. очень долго до этого...,2.0,yandex
9757,70000001091778279,"расположен на 4 этаже трц калейдоскоп, на фудк...","ассортимент, известная специфика, сетевой рест...",вкусно — и точка,грузинский ресторан,Москва,"Сходненская улица, 56",55.850499,37.444422,4.0,1,0.0,NaN,"расположен на 4 этаже трц калейдоскоп, на фудк...",4.0,yandex
9758,70000001091778280,"в целом вкусно, но раньше шаурмист прям большу...","большая шаурма, вкусная шаурма, вкусно, лагман",душа лаваша,ресторан,Москва,"Краснопролетарская улица, 16с1",55.777089,37.607197,4.0,1,0.0,NaN,"в целом вкусно, но раньше шаурмист прям большу...",4.0,yandex
9759,70000001091778281,бронировали vip комнату 4.02 на 6 человек и не...,"10% скидка, безупречное обслуживание, беспрово...",караоке-клуб маска,ресторан,Москва,"улица Маросейка, 7/8",55.757592,37.632880,5.0,5,0.0,NaN,бронировали vip комнату 4.02 на 6 человек и не...,5.0,yandex


In [26]:
dff.drop(columns=["review"], inplace=True)

In [27]:
dff.rename(columns={"combined_review": "reviews"}, inplace=True)
dff

,place_id,reviews,features,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,rating,source
0,4504127908348140,"в принципе, нормальный ресторанчик при отеле. ...","бизнес-ланч, биточки, вкусная еда, гречотто, к...",495,пивоваренный ресторан,Москва,"Олимпийский проспект, 18/1",55.785009,37.624123,5.0,1,0.0,1500.0,5.0,flamp
1,4504127908348160,"есть такой ресторан на юге москвы, по названию...","бизнес-ланчи, вкусная еда, вкусный стейк, дело...",Bora bora cafe,ресторан,Москва,"Ореховый бульвар, 14к3",55.609493,37.719954,4.0,3,1.0,1500.0,5.0,flamp
2,4504127908348342,прекрасные завтраки от 195 р. есть постные блю...,"завтраки, кофе с соевым молоком, милые надписи...",Coffee,кофейня,Москва,"Душинская улица, 7с1",55.745250,37.706002,5.0,1,0.0,1500.0,5.0,flamp
3,4504127908348384,крейзи! именно так можно сказать о данном заве...,"бесплатные концерты, веселая атмосфера, винило...",Bourbon street,пивной ресторан,Москва,"Большой Златоустинский переулок, 7с1",55.758709,37.632634,4.9,5,0.0,NaN,5.0,flamp
4,4504127908348455,уютный семейный итальянский ресторан с плетено...,"арматная, атмосфера, баклажаны, буррата, зелен...",Cantinetta Antinori,ресторан,Москва,"Денежный переулок, 20",55.745428,37.586787,5.0,1,0.0,5000.0,5.0,flamp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9756,70000001091778278,"заказала тортик, очень вкусный, свежий, заказы...","вкуснейший, вкусные десерты, вкусный, вкусняшк...",особенный наполеон,ресторан,Москва,"Мосфильмовская улица, 53",55.704864,37.497450,2.0,7,0.0,NaN,2.0,yandex
9757,70000001091778279,"расположен на 4 этаже трц калейдоскоп, на фудк...","ассортимент, известная специфика, сетевой рест...",вкусно — и точка,грузинский ресторан,Москва,"Сходненская улица, 56",55.850499,37.444422,4.0,1,0.0,NaN,4.0,yandex
9758,70000001091778280,"в целом вкусно, но раньше шаурмист прям большу...","большая шаурма, вкусная шаурма, вкусно, лагман",душа лаваша,ресторан,Москва,"Краснопролетарская улица, 16с1",55.777089,37.607197,4.0,1,0.0,NaN,4.0,yandex
9759,70000001091778281,бронировали vip комнату 4.02 на 6 человек и не...,"10% скидка, безупречное обслуживание, беспрово...",караоке-клуб маска,ресторан,Москва,"улица Маросейка, 7/8",55.757592,37.632880,5.0,5,0.0,NaN,5.0,yandex


In [33]:
dff.to_csv("features_nona.csv", index=False)

In [42]:
test = df.copy()

test["review_length"] = test["review"].apply(len)

In [38]:
grouped_stats = test.groupby("source")["review_length"].describe()
grouped_stats

,count,mean,std,min,25%,50%,75%,max
source,,,,,,,,
flamp,23127.0,421.914688,452.478034,82.0,175.0,269.0,484.0,7997.0
yandex,24707.0,294.361112,270.476964,6.0,142.0,211.0,354.0,4793.0


In [40]:
test[test["review_length"] < 50]

,place_id,name_primary,name_extension,city,address,lat,lon,rating_decimal,reviews_count,business_lunch,avg_price,review,rating,source,review_length
1052,4504127908387776,Колбасофф,ресторан,Москва,"Симферопольский бульвар, 22к2",55.647452,37.604762,4.0,7,1.0,2000.0,вкусно и атмосферно. особенно когда живая музыка,4.0,yandex,48
1306,4504127908408218,Пикассо,кафе-бар,Москва,"улица Миклухо-Маклая, 15",55.652705,37.505355,5.0,5,1.0,1500.0,"хороший ресторан, приятный коллектив",5.0,yandex,36
1505,4504127908427102,Шоколадница,кофейня,Москва,"Краснопрудная улица, 3-5с1",55.777646,37.660952,4.0,8,0.0,NaN,средненько...,4.0,yandex,13
2903,4504127908549261,Geraldine,необистро,Москва,"улица Остоженка, 27к2",55.739042,37.597199,5.0,23,0.0,2500.0,один из лучших ресторанов москвы !,5.0,yandex,34
3453,4504127908652925,Вкусно — и точка,ресторан быстрого питания,Москва,"Каширское шоссе, 61/15",55.612217,37.718121,4.0,8,0.0,350.0,очень много народа. не хватает мест в зале.,4.0,yandex,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47756,70000001091778248,мята lounge,ресторан,Москва,"Новокузнецкая улица, 11/13",55.738282,37.631586,5.0,3,0.0,NaN,"классный бар,уютный,прекрасный персонал",5.0,yandex,39
47764,70000001091778253,booblik,ресторан,Москва,"Северный административный округ, улица 8 Марта...",55.802755,37.555984,5.0,5,0.0,NaN,любимая кофейня. вкусная выпечка и напитки.,5.0,yandex,43
47778,70000001091778262,holy berry,ресторан,Москва,"Костомаровский переулок, 3с12",55.752197,37.664941,5.0,1,0.0,NaN,лучший кофей на р-не и цены оч демократичные,5.0,yandex,44
47807,70000001091778272,hayot,ресторан,Москва,"Волгоградский проспект, 73",55.710073,37.745187,5.0,1,0.0,NaN,просто супер полов оченннн вкусно,5.0,yandex,33
